In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_exclude_dr 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
SELECT 
    store_id,
    product_id, 
    note,
    start_date,
    end_date
FROM (
    SELECT 
        B.*,
        ROW_NUMBER() OVER (PARTITION BY store_id,product_id,end_date ORDER BY proc_date) RN
    FROM (
        SELECT 
            A.*, 
            DENSE_RANK() OVER (PARTITION BY store_id,product_id ORDER BY prior) RN1
        FROM (
            SELECT  
                a.product_id, b.store_id, note, proc_date, start_date, end_date, 2 prior
                FROM {catalog_name}.udp_wcm_silver_dim.stg_vm_dr_exclude_by_all a 
                CROSS JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b 
                WHERE b.business_unit = '1500'
            UNION ALL
            SELECT 
                product_id, site store_id, note, proc_date, start_date, end_date, 3 prior 
                FROM {catalog_name}.udp_wcm_silver_dim.stg_vm_dr_exclude_by_site
            UNION ALL
            SELECT 
                product_id, b.store_id, note, proc_date, start_date, end_date, 4 prior 
                FROM {catalog_name}.udp_wcm_silver_dim.stg_vm_dr_exclude_by_region a
                LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b ON b.region_domain = a.region AND business_unit = '1500'
            UNION ALL
            SELECT  
                a.product_id, b.store_id, abc note, proc_date, start_date, end_date, 5 prior
                FROM {catalog_name}.udp_wcm_silver_dim.stg_vm_hangthoivu a 
                CROSS JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b 
                WHERE b.business_unit = '1500'
            UNION ALL
            SELECT  
                pr.product_id, b.store_id, NULL note, a.proc_date, a.start_date, a.end_date, 1 prior
                FROM {catalog_name}.udp_wcm_silver_dim.stg_vm_dr_exclude_by_mch5 a
                INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product pr ON a.mch5_id = pr.mch5_id
                CROSS JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b 
                WHERE b.business_unit = '1500'
            UNION ALL
            SELECT  
                pr.product_id, b.store_id, NULL note, a.proc_date, a.start_date, a.end_date, 1 prior
                FROM {catalog_name}.udp_wcm_silver_dim.stg_vm_dr_exclude_by_mch3 a
                INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product pr ON a.mch3_id = pr.mch3_id
                CROSS JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b 
                WHERE b.business_unit = '1500'
        ) A
    ) B WHERE B.RN1 = 1
) X WHERE X.RN = 1
;
""")